In [1]:
!mkdir -p ~/agave/funwave-tvd-docker

%cd ~/agave

!pip3 install setvar

import re
import os
import sys
from setvar import *
from time import sleep

# This cell enables inline plotting in the notebook
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
loadvar()
!auth-tokens-refresh

/home/jovyan/agave
AGAVE_JSON_PARSER=jq
AGAVE_PASSWD=**HIDDEN**
AGAVE_USERNAME=stevenrbrandt
APP_NAME=funwave-tvd-jetstream-stevenrbrandt
BASE_APP_NAME=funwave-tvd
CMD=jobs-output-get 6351097811171274265-242ac11b-0001-007 fork-command-1.out
DEPLOYMENT_PATH=agave-deployment
DOCKERHUB_NAME=stevenrbrandt
DOMAIN=jetstream-cloud.org
EMAIL=sbrandt@cct.lsu.edu
EXEC_MACHINE=jetstream-exec-stevenrbrandt
HOME_DIR=/home/jovyan
INPUTS={"datafile":"agave://jetstream-storage-stevenrbrandt/input.txt"}
IN_FILE=agave://jetstream-storage-stevenrbrandt/funwave-tvd-docker/dockerjob.tgz
JOB_FILE=job-remote-31140.txt
JOB_ID=6351097811171274265-242ac11b-0001-007
MACHINE_IP=149.165.157.140
MACHINE_NAME=jetstream
MACHINE_USERNAME=jovyan
OUTPUT=Successfully submitted job 6351097811171274265-242ac11b-0001-007
PATH=/home/jovyan/agave/cli/bin:/opt/conda/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/local/jq/bin:/home/jovyan/src/cli/bin
PBTOK=**HIDDEN**
PORT=10022
REMOTE_COMMA

<h2>Creating the Docker Image</h2>
To start, we need a Dockerfile, which has a number of simple commands.
It starts with "FROM" which can specify any docker image available from DockerHub. That not only includes basic operating systems such as "ubunto", "fedora", "centos", etc. but specialized containers made by anyone with a dockerhub account. I've provided "science-base" which has OpenMPI 2.1.1 and some standard compilers, i.e. gfortran, gcc, and g++.

MAINTAINER is a bit of metadata that (hopefully) will allow you to contact the container's creator, if need be.

WORKDIR is the dockerfile equivalent of the "cd" command. Note that running "cd" will not change your directory.

RUN simply runs the command that follows. Because the container is saved after each step, we want to avoid creating files that we don't want to keep (we want containers to be as small as possible).

USER specifies the user id for running subsequent RUN commands.

COPY can be used to copy files into the container from the build directory.

ENTRYPOINT is a script that runs when the container starts up. What our script does is create a new user on the docker image with a user id and name that is convenient.

In [3]:
writefile("funwave-tvd-docker/Dockerfile","""
FROM stevenrbrandt/science-base

MAINTAINER Steven R. Brandt <sbrandt@cct.lsu.edu>
RUN cd && \
    git clone https://github.com/fengyanshi/FUNWAVE-TVD && \
    cd FUNWAVE-TVD/src && \
    perl -p -i -e 's/FLAG_8 = -DCOUPLING/#$&/' Makefile && \
    make

WORKDIR /home/jovyan/FUNWAVE-TVD/src
RUN mkdir -p /home/jovyan/FUNWAVE-TVD/rundir
WORKDIR /home/jovyan/FUNWAVE-TVD/rundir
USER root
COPY entrypoint.sh /usr/local/bin
RUN chmod 755 /usr/local/bin/entrypoint.sh
ENTRYPOINT ["/usr/local/bin/entrypoint.sh"]
""")

Writing file `funwave-tvd-docker/Dockerfile'


This is out entrypoint file. It can be any bash script, and it is not required.

In [2]:
writefile("funwave-tvd-docker/entrypoint.sh","""
#!/bin/bash
if [ "\$USER_ID" != "" ]
then
  if [ "\$USER_NAME" = "" ]
  then
    USER_NAME=user
  fi
  useradd --shell /bin/bash -u \$USER_ID -o -p ABadPassword -c "" -m \$USER_NAME
  export HOME=/home/\$USER_NAME

  if [ "$@" = "bash" ]
  then
    exec su -m - \$USER_NAME
  else
    exec su -m - \$USER_NAME -c "$@"
  fi
else
  if [ "$@" = "bash" ]
  then
    exec bash
  else
    exec bash -c "$@"
  fi
fi
""")




writefile("funwave-tvd-docker/entrypoint.sh","""
#!/bin/bash
echo ARGS: \$@
/home/jovyan/FUNWAVE-TVD/src/funwave_vessel "\$@"
""")

Writing file `funwave-tvd-docker/entrypoint.sh'


Now that we've create our Dockerfile and entrypoint.sh, bundle them up in a tarball and send them somewhere that agave can access them.

In [4]:
!tar -czf dockerjob.tgz -C funwave-tvd-docker Dockerfile entrypoint.sh
!files-mkdir -S ${STORAGE_MACHINE} -N funwave-tvd-docker
!files-upload -F dockerjob.tgz -S ${STORAGE_MACHINE} funwave-tvd-docker/

Successfully created folder funwave-tvd-docker
Uploading dockerjob.tgz...
######################################################################## 100.0%


In [5]:
import runagavecmd as r
import imp
imp.reload(r)

<module 'runagavecmd' from '/home/jovyan/agave/runagavecmd.py'>

Run the docker build command. We will "tag" this build with the name "funwave-tvd" when it is complete.

In [6]:
r.runagavecmd(
    "tar xzf dockerjob.tgz && sudo docker build --rm -t funwave-tvd .",
    "agave://${STORAGE_MACHINE}/funwave-tvd-docker/dockerjob.tgz"
)

REMOTE_COMMAND=tar xzf dockerjob.tgz && sudo docker build --rm -t funwave-tvd .
INPUTS={"datafile":"agave://jetstream-storage-stevenrbrandt/funwave-tvd-docker/dockerjob.tgz"}
JOB_FILE=job-remote-1113.txt
Writing file `job-remote-1113.txt'
OUTPUT=Successfully submitted job 7320503265959800345-242ac11b-0001-007
JOB_ID=7320503265959800345-242ac11b-0001-007
STAT=PENDING
STAT=PENDING
STAT=PROCESSING_INPUTS
STAT=STAGED
STAT=STAGED
STAT=SUBMITTING
STAT=SUBMITTING
STAT=RUNNING
STAT=FINISHED
CMD=jobs-output-get 7320503265959800345-242ac11b-0001-007 fork-command-1.out
All done! Output follows.
Reading file `fork-command-1.out'
Sending build context to Docker daemon  13.82kB

Step 1/10 : FROM stevenrbrandt/science-base
 ---> e93db47f971c
Step 2/10 : MAINTAINER Steven R. Brandt <sbrandt@cct.lsu.edu>
 ---> Using cache
 ---> 9bcc1f7703a9
Step 3/10 : RUN cd &&     git clone https://github.com/fengyanshi/FUNWAVE-TVD &&     cd FUNWAVE-TVD/src &&     perl -p -i -e 's/FLAG_8 = -DCOUPLING/#$&/' Makefile &

In [7]:
!jobs-output-get ${JOB_ID} fork-command-1.err
!cat fork-command-1.err

<h2>Running the Docker Image</h2>
It is possible to run docker interactively, but that isn't convenient inside scripts. So instead, we start it in detached mode, with the -d flag.

Because your docker image has its own internal file system, it can't see files on the host machine. You can, however, transfer them using the "docker cp" command.

Running docker is slightly tricky. When a Docker image starts up, you can execute any command you want--but when you type "exit" all the changes you've made to the file system vanish. Therefore it's necessary to copy them out before the docker container stops.

In [25]:
writefile("rundock.sh","""
rm -fr cid.txt out.tgz

# Start a docker image running in detached mode, write the container id to cid.txt
sudo docker run -d -it --rm --cidfile cid.txt -e USER_ID=$(id -u) -e USER_NAME=$USER stevenrbrandt/funwave-tvd bash

# Store the container id in CID for convenience
CID=\$(cat cid.txt)

# Copy the input.txt file into the running image
sudo docker cp input.txt \$CID:/home/jovyan/FUNWAVE-TVD/rundir/

# Run funwave on the image
sudo docker exec --user jovyan \$CID mpirun -np 4 /home/jovyan/FUNWAVE-TVD/src/funwave_vessel

# Extract the output files from the running image
# Having them in a tgz makes it more convenient to fetch them with jobs-output-get
sudo docker exec --user jovyan \$CID tar czf -  output > out.tgz

# Stop the image
sudo docker stop \$CID

# List the output files
tar tzf out.tgz
""")

Writing file `rundock.sh'


Upload the input.txt file and the rundock.sh script.

In [12]:
!tar czf rundock.tgz rundock.sh input.txt
!files-upload -F rundock.tgz -S ${STORAGE_MACHINE} funwave-tvd-docker/

Uploading rundock.tgz...
######################################################################## 100.0%


Execute the rundock.sh script

In [26]:
r.runagavecmd(
    "tar xzf rundock.tgz && bash rundock.sh",
    "agave://${STORAGE_MACHINE}/funwave-tvd-docker/rundock.tgz")

REMOTE_COMMAND=tar xzf rundock.tgz && bash rundock.sh
INPUTS={"datafile":"agave://jetstream-storage-stevenrbrandt/funwave-tvd-docker/rundock.tgz"}
JOB_FILE=job-remote-1113.txt
Writing file `job-remote-1113.txt'
OUTPUT=Successfully submitted job 1989734924979596825-242ac11b-0001-007
JOB_ID=1989734924979596825-242ac11b-0001-007
STAT=PENDING
STAT=PENDING
STAT=PENDING
STAT=STAGING_INPUTS
STAT=STAGED
STAT=SUBMITTING
STAT=SUBMITTING
STAT=RUNNING
STAT=RUNNING
STAT=RUNNING
STAT=RUNNING
STAT=RUNNING
STAT=RUNNING
STAT=FINISHED
CMD=jobs-output-get 1989734924979596825-242ac11b-0001-007 fork-command-1.out
All done! Output follows.
Reading file `fork-command-1.out'
b8dd7f9a0019ecd307add02c0493f35a4caa2cacd13e78aaee474be1d201caae
input.txt
/home/jovyan/FUNWAVE-TVD/rundir
 BATHY_CORRECTION DOES NOT EXIST. USE DEFAULT VALUE
 INI_UVZ DOES NOT EXIST. USE DEFAULT VALUE
 WaterLevel DOES NOT EXIST. USE DEFAULT VALUE
 ETA_LIMITER DOES NOT EXIST. USE DEFAULT VALUE
 EqualEnergy DOES NOT EXIST. USE DEFAULT VALU

Get the output of the job back to our local machine

In [16]:
!jobs-output-list ${JOB_ID}
!jobs-output-get ${JOB_ID} out.tgz
!tar xzvf out.tgz

.agave.archive
.agave.log
cid.txt
fork-command-1.err
fork-command-1.ipcexe
fork-command-1.out
fork-command-1.pid
fork-test.txt
fork-wrapper.txt
input.txt
out.tgz
rundock.sh
rundock.tgz
######################################################################## 100.0%
output/
output/eta_00010
output/u_00008
output/u_00004
output/v_00007
output/u_00009
output/eta_00011
output/u_00007
output/u_00001
output/v_00001
output/eta_00009
output/eta_00004
output/u_00010
output/u_00006
output/u_00011
output/v_00003
output/eta_00006
output/eta_00002
output/eta_00005
output/v_00009
output/v_00008
output/v_00006
output/u_00002
output/v_00011
output/u_00005
output/v_00004
output/u_00003
output/v_00010
output/eta_00001
output/eta_00007
output/eta_00003
output/v_00002
output/eta_00008
output/v_00005


<h2>Running with Singularity</h2>
If we have a public docker image, we can run it directly with Singularity. Singularity is desiged to be more HPC friendly than Docker. First, because it doesn't all the running user to access any user id but their own inside the container, and second, because singularity images can be run through MPI, making it easier to scale up to a distributed cluser.

In this first step, we build the singularity installation. Because the result of this job is intended to be an installation for subsequent jobs, we install it to a hard-coded directory rather than using the normal Agave job directory.

In [17]:
!files-mkdir -S ${STORAGE_MACHINE} -N sing
!files-upload -F input.txt -S ${STORAGE_MACHINE} sing/
r.runagavecmd(
            "mkdir -p ~/singu && "+
            "cd ~/singu && "+
            "rm -f funwave-tvd.img && "+
            "singularity create funwave-tvd.img --size 2000 && "+
            "singularity import funwave-tvd.img docker://stevenrbrandt/funwave-tvd:latest")

Successfully created folder sing
Uploading input.txt...
######################################################################## 100.0%
REMOTE_COMMAND=mkdir -p ~/singu && cd ~/singu && rm -f funwave-tvd.img && singularity create funwave-tvd.img --size 2000 && singularity import funwave-tvd.img docker://stevenrbrandt/funwave-tvd:latest
INPUTS={}
JOB_FILE=job-remote-1113.txt
Writing file `job-remote-1113.txt'
OUTPUT=Successfully submitted job 3547179470548692505-242ac11b-0001-007
JOB_ID=3547179470548692505-242ac11b-0001-007
STAT=PENDING
STAT=PENDING
STAT=STAGED
STAT=STAGED
STAT=SUBMITTING
STAT=SUBMITTING
STAT=RUNNING
STAT=RUNNING
STAT=RUNNING
STAT=RUNNING
STAT=FINISHED
CMD=jobs-output-get 3547179470548692505-242ac11b-0001-007 fork-command-1.out
All done! Output follows.
Reading file `fork-command-1.out'
Initializing Singularity image subsystem
Opening image file: funwave-tvd.img
Creating 768MiB image
Binding image to loop
Creating file system within image
Image is done: funwave-tvd.img
D

Now that the Singularity image is built, we can run it with mpi. Notice that mpi executes the singularity command. The tricky part here is to make sure you've got the same version of mpi running inside and outside the container.

In [22]:
!files-upload -F input.txt -S ${STORAGE_MACHINE} ./
r.runagavecmd(
    "export LD_LIBRARY_PATH=/usr/local/lib && "+
    "mpirun -np 4 singularity exec ~/singu/funwave-tvd.img /home/jovyan/FUNWAVE-TVD/src/funwave_vessel && "+
    "tar cvzf singout.tgz output",
    "agave://${STORAGE_MACHINE}/input.txt"
)

Uploading input.txt...
######################################################################## 100.0%
REMOTE_COMMAND=export LD_LIBRARY_PATH=/usr/local/lib && mpirun -np 4 singularity exec ~/singu/funwave-tvd.img /home/jovyan/FUNWAVE-TVD/src/funwave_vessel && tar cvzf singout.tgz output
INPUTS={"datafile":"agave://jetstream-storage-stevenrbrandt/input.txt"}
JOB_FILE=job-remote-1113.txt
Writing file `job-remote-1113.txt'
OUTPUT=Successfully submitted job 7085061267691138585-242ac11b-0001-007
JOB_ID=7085061267691138585-242ac11b-0001-007
STAT=PENDING
STAT=PENDING
STAT=PENDING
STAT=STAGING_INPUTS
STAT=STAGED
STAT=STAGED
STAT=SUBMITTING
STAT=RUNNING
STAT=RUNNING
STAT=RUNNING
STAT=RUNNING
STAT=RUNNING
STAT=FINISHED
CMD=jobs-output-get 7085061267691138585-242ac11b-0001-007 fork-command-1.out
All done! Output follows.
Reading file `fork-command-1.out'
 BATHY_CORRECTION DOES NOT EXIST. USE DEFAULT VALUE
 INI_UVZ DOES NOT EXIST. USE DEFAULT VALUE
 WaterLevel DOES NOT EXIST. USE DEFAULT VALUE
 ET

In [24]:
!jobs-output-get ${JOB_ID} singout.tgz
!rm -fr output
!tar xzf singout.tgz

######################################################################## 100.0%
